# Scrape the content of p tags in these links

### Import libraries & links dataset

In [5]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
from time import sleep
import pandas as pd
import random

In [6]:
!pwd

/Users/kaankorkmaz/projects_KK/thesis/scraping/wikipedia


In [7]:
df = pd.read_csv('/Users/kaankorkmaz/projects_KK/thesis/data/german_medical_wiki/list_links2scrape.csv', index_col=0)

In [8]:
df.head()

,links
0,https://de.wikipedia.org/wiki/Berufskrankheit
1,https://de.wikipedia.org/wiki/Aluminose
2,https://de.wikipedia.org/wiki/Alveolitis
3,https://de.wikipedia.org/wiki/Anthrakosilikose
4,https://de.wikipedia.org/wiki/Asbestose


In [9]:
links = df["links"]

### Data collection

In [10]:
# Such loop is necessary to overcome the ConnectionError

all_text = []
start = 0
count = 0

while start < 2766:
    
    start = 0 + count
    
    for url in tqdm(links[start:], position=0, leave=True):
        try:
            count+=1
            
            page = requests.get(url)
            soup = BeautifulSoup(page.content, 'html.parser')
            p_s = soup.find_all('p')
            p_s_text = [curr.text for curr in p_s]
            all_text.append(p_s_text)
        except:
            print('Stopped at {}'.format(count))
            continue

100%|██████████| 2766/2766 [17:46<00:00,  2.59it/s]
0it [00:00, ?it/s]


In [13]:
len(all_text)

2766

### Concatanation

In [20]:
flat_all_text = [item for sublist in all_text for item in sublist]
flat_all_text_concat = [item for word in flat_all_text for item in word.split()]
flat_all_text_concat_by_sentence = [item for word in flat_all_text for item in word.split('.')]

In [16]:
len(flat_all_text)

31479

In [18]:
flat_all_text[0]

'Eine Berufskrankheit ist eine Krankheit, die durch die berufliche (versicherte) Tätigkeit verursacht worden ist und nach dem jeweils geltenden Recht auch formal als Berufskrankheit anerkannt ist. Typische Berufskrankheiten sind Lärmschwerhörigkeit, Hautkrankheiten, Erkrankungen des Stütz- und Bewegungsapparats sowie Erkrankungen durch anorganische Stäube (Asbestose und Silikose). Psychische Erschöpfungszustände, wie das Burnout-Syndrom, psychische Störungen oder psychiatrische Erkrankungen zählen bislang nicht zu den Berufskrankheiten.\n'

In [15]:
len(flat_all_text_concat)

1425168

In [19]:
flat_all_text_concat[0]

'Eine'

In [21]:
len(flat_all_text_concat_by_sentence)

122830

In [29]:
flat_all_text_concat_by_sentence_cleaned = [good_sentence for good_sentence in flat_all_text_concat_by_sentence if len(good_sentence) > 50]

In [30]:
len(flat_all_text_concat_by_sentence_cleaned)

80220

In [31]:
flat_all_text_concat_by_sentence_cleaned[0:5]

['Eine Berufskrankheit ist eine Krankheit, die durch die berufliche (versicherte) Tätigkeit verursacht worden ist und nach dem jeweils geltenden Recht auch formal als Berufskrankheit anerkannt ist',
 ' Typische Berufskrankheiten sind Lärmschwerhörigkeit, Hautkrankheiten, Erkrankungen des Stütz- und Bewegungsapparats sowie Erkrankungen durch anorganische Stäube (Asbestose und Silikose)',
 ' Psychische Erschöpfungszustände, wie das Burnout-Syndrom, psychische Störungen oder psychiatrische Erkrankungen zählen bislang nicht zu den Berufskrankheiten',
 'Mehr als zwanzigtausend Menschen erkranken jährlich in den deutschsprachigen Staaten an Berufskrankheiten',
 ' Sie werden von den Unfallversicherungsträgern medizinisch rehabilitiert und finanziell entschädigt']

In [32]:
random.shuffle(flat_all_text_concat_by_sentence_cleaned)

In [33]:
flat_all_text_concat_by_sentence_cleaned[0:5]

[' Die Beweglichkeit der Zunge ist eingeschränkt, so dass sie beispielsweise nicht über die untere Zahnleiste, geschweige denn über die Unterlippe hinausgestreckt werden kann',
 ' 1700 beschrieb Bernardino Ramazzini die durch Allergene entstandenen Berufskrankheiten in De morbis artificum diatriba',
 ' Entsprechendes wird weltweit immer wieder berichtet',
 'Da taube Personen durch ihre Kommunikationsbehinderung in der Gesellschaft häufig isoliert sind, werden soziale Kontakte gern innerhalb von Gehörlosenkreisen gepflegt',
 ' Beispiele für im Stoffwechsel vorkommende aggressive Oxidantien sind Sauerstoffradikale, Hydroxyl-Radikale und Wasserstoffperoxid']

In [26]:
!pwd

/Users/kaankorkmaz/projects_KK/thesis/scraping/wikipedia


### All sentences

In [34]:
textfile = open("/Users/kaankorkmaz/projects_KK/thesis/data/german_medical_wiki/gmw-sentences.txt", "w")
for element in flat_all_text_concat_by_sentence_cleaned:
    textfile.write(element + "\n")
textfile.close()

### Train set

In [39]:
total_items = len(flat_all_text_concat_by_sentence_cleaned)
total_items

80220

In [46]:
train_start_index = int(total_items * 0.2 + 1)
test_finish_index = int(train_start_index - 1)
print(f"Train start index:{train_start_index}")
print(f"Test finish index:{test_finish_index}")

Train start index:16045
Test finish index:16044


In [47]:
test_finish_index 

16044

In [48]:
textfile = open("/Users/kaankorkmaz/projects_KK/thesis/data/german_medical_wiki/gmw-sentences_train.txt", "w")
for element in flat_all_text_concat_by_sentence_cleaned[train_start_index:]:
    textfile.write(element + "\n")
textfile.close()

### Test set

In [49]:
textfile = open("/Users/kaankorkmaz/projects_KK/thesis/data/german_medical_wiki/gmw-sentences_test.txt", "w")
for element in flat_all_text_concat_by_sentence_cleaned[0:test_finish_index]:
    textfile.write(element + "\n")
textfile.close()

### Dataframe creation & save as CSV

In [63]:
df_all_text = pd.DataFrame(flat_all_text_concat)

In [68]:
df_all_text

,0
0,Eine
1,Berufskrankheit
2,ist
3,eine
4,"Krankheit,"
...,...
1413317,dieser
1413318,"Kategorie,"
1413319,von
1413320,80


In [69]:
df_all_text.to_csv('data/vocabulary/germanwiki_vocab.csv', index=False)